# Лабораторная работа №5. <br> Метрики качества классификации

Выполнила: Кузнецова Екатерина

## 1. Цель работы

Рассмотреть различные метрики качества классификации, входящих в состав библиотеки scikit-learn

## 2. Постановка задачи

1. Прочитать теоретическую часть
Для двух любых методов классификации из предыдущих работ и своего набора данных посчитать следующие метрики качества:<br>
	a. Точность классификации (Classification Accuracy)<br>
	b. Логарифм функции правдоподобия (Logarithmic Loss)<br>
	c. Область под кривой ошибок (Area Under ROC Curve)<br>
	d. Матрица неточностей (Confusion Matrix)<br>
	e. Отчет классификации (Classification Report)<br>
2. Для более точных результатов использовать кросс-валидацию
3. Сравнить применимость используемых классификаторов, основываясь на полученных метриках

## 3. Исходные данные

Датасет: http://archive.ics.uci.edu/ml/datasets/Zoo<br>
Предметная область: обитатели зоопарка<br>
Задача: определить, к какому из 7ми типов относится каждое животное

Количество записей: 101<br>
Количество атрибутов: 17


Атрибуты:

1. Название животного (строка, уникальный для каждого экземпляра)
2. Наличие волос (логический тип)
3. Наличие перьев (логический тип)
4. Яйца (логический тип)
5. Млекопитающий (логический тип)
6. Умеет летать (логический тип)
7. Водный (логический тип)
8. Хищник (логический тип)
9. Наличие зубов (логический тип) 
10. Наличие позвоночника (логический тип)
11. Дышит воздухом (логический тип)
12. Ядовитость (логический тип)
13. Наличие плавников (логический тип)
14. Количество ног (набор целочисленных значений: {0,2,4,5,6,8})
15. Наличие хвоста (логический тип)
16. Является домашним (логический тип)
17. Catsize (логический тип)
18. Тип (целочисленные значения в диапазоне [1,7])

## 4. Ход работы

Загружаем датасет и делим его на 2 части: animal_attr, где хранятся значения атрибутов каждого объекта, animal_class - массив классов, к которым принадлежат эти объекты. Далее разбиваем выборку на тестовую и обучающую

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("zoo.data.csv", header=None).values
animal_attr = dataset[:, 1:-1]  # список атрибутов (признаков) для каждого животного
animal_class = dataset[:, -1]  # классы животных
animal_class = animal_class.astype(np.int64, copy=False)
X_train, X_test, Y_train, Y_test = train_test_split(animal_attr, animal_class, test_size=0.35)

Посчитаем метрики качества для Наивного Байесовского классификатора и метода К ближайших соседей. Для более точных результатов будем использовать кросс-валидацию.

In [7]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold, cross_val_score

# функция для подсчета метрик
def calc_metric(estimator):
    # для каждого метода классификации
    print('Классификатор: {}'.format(estimator))
    # вычисляем точность классификации и логарифм функции правдоподобия
    for scoring in ['accuracy', 'neg_log_loss']:
        result = cross_val_score(estimator, animal_attr, animal_class, cv=KFold(n_splits=2, shuffle=True),
                                 scoring=scoring)
        print(' Метрика: {}. Точность: {:.3f} ({:.3f})'.format(scoring, result.mean(), result.std()))
    
    # обучаем классификатор
    estimator.fit(X_train, Y_train)
    # классифицируем тестовую выборку
    predicted = estimator.predict(X_test)

    # вычисление матрицы неточностей
    print(' Метрика: Confusion Matrix\n {}'.format(confusion_matrix(Y_test, predicted)))
    # отчет классификации
    print(' Метрика: Classification Report\n {}'.format(classification_report(Y_test, predicted)))

ROC-кривая позволяет оценивать качество только бинарной классификации, а в нашем случае третуется разбиение данных по 7 классам. Поэтому в данном случае оценка "Area Under ROC Curve" не применима.

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

calc_metric(GaussianNB())
calc_metric(KNeighborsClassifier(n_neighbors=25))

Классификатор: GaussianNB(priors=None)
 Метрика: accuracy. Точность: 0.960 (0.020)
 Метрика: neg_log_loss. Точность: -1.700 (1.009)
 Метрика: Confusion Matrix
 [[12  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0  0  7  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  0  0  3  0]
 [ 0  0  0  0  0  0  4]]
 Метрика: Classification Report
              precision    recall  f1-score   support

          1       1.00      1.00      1.00        12
          2       1.00      1.00      1.00         8
          3       1.00      1.00      1.00         1
          4       1.00      1.00      1.00         7
          5       1.00      1.00      1.00         1
          6       1.00      1.00      1.00         3
          7       1.00      1.00      1.00         4

avg / total       1.00      1.00      1.00        36

Классификатор: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=25, p=2,
        

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 5. Заключение

В ходе лабораторной работы были посчитаны 4 метрики качества для Наивного Байесовского классификатора и метода К ближайших соседей. Метрика Area Under ROC Curve оказалась неприменимой к датасету "Зоопарк".

Точность классификации GaussianNB намного выше, чем у KNeighborsClassifier. При анализе матрицы неточностей и отчета классификации было выявлено, что алгоритм К ближайших соседей ошибается намного чаще. KNeighborsClassifier выигрывает только по метрике Logarithmic Loss. Из этого можно сделать вывод, что байесовский алгоритм лучше подходит для данного датасета. 